In [389]:
import numpy as np
import csv

import sys
sys.path.append('../../')
import pyGM as gm;
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  
with open("../league of legends/cut_data/lol_cut.csv", newline='') as i:
    reader = csv.reader(i)
    lol_data= list(reader)

In [390]:
for i in range(5):
    for j in lol_data[i]:
        print(j)

gameId
creationTime
gameDuration
seasonId
winner
firstBlood
firstTower
firstInhibitor
firstBaron
firstDragon
firstRiftHerald
3326086514
1504279457970
1949
9
1
2
1
1
1
1
2
3229566029
1497848803862
1851
9
1
1
1
1
0
1
1
3327363504
1504360103310
1493
9
1
2
1
1
1
2
0
3326856598
1504348503996
1758
9
1
1
1
1
1
1
0


# turn the data into something managable by pygm

In [391]:
#using 6 combinations, one for each first objective


first_combinations=7
nlevels=10
scale=.2

X = [None]*first_combinations
for i in range(first_combinations):
    X[i] = gm.Var(i, nlevels)  

del lol_data[0] #delete the header to avoid fuck ups
Pwin = np.zeros( (nlevels,nlevels) )
#pr(win | first objectives)
for i in range(nlevels):
    for j in range(nlevels):
        diff = i-j                   # find the advantage of Pi over Pj, then 
        Pwin[i,j] = (1./(1+np.exp(-scale*diff)))  # Pwin = logistic of advantage
print(Pwin)


factors = [ gm.Factor([X[i]],1./nlevels) for i in range(first_combinations) ] #uniform belief over team skill
    

[[0.5        0.450166   0.40131234 0.35434369 0.31002552 0.26894142
  0.23147522 0.19781611 0.16798161 0.14185106]
 [0.549834   0.5        0.450166   0.40131234 0.35434369 0.31002552
  0.26894142 0.23147522 0.19781611 0.16798161]
 [0.59868766 0.549834   0.5        0.450166   0.40131234 0.35434369
  0.31002552 0.26894142 0.23147522 0.19781611]
 [0.64565631 0.59868766 0.549834   0.5        0.450166   0.40131234
  0.35434369 0.31002552 0.26894142 0.23147522]
 [0.68997448 0.64565631 0.59868766 0.549834   0.5        0.450166
  0.40131234 0.35434369 0.31002552 0.26894142]
 [0.73105858 0.68997448 0.64565631 0.59868766 0.549834   0.5
  0.450166   0.40131234 0.35434369 0.31002552]
 [0.76852478 0.73105858 0.68997448 0.64565631 0.59868766 0.549834
  0.5        0.450166   0.40131234 0.35434369]
 [0.80218389 0.76852478 0.73105858 0.68997448 0.64565631 0.59868766
  0.549834   0.5        0.450166   0.40131234]
 [0.83201839 0.80218389 0.76852478 0.73105858 0.68997448 0.64565631
  0.59868766 0.549834  

In [392]:
print(len(X))

7


In [393]:
#turn the csv into str because oh my god csv's are all strings thats so stupid
for i in range(len(lol_data)):
    for j in range(len(lol_data[i])):
        lol_data[i][j]=int(lol_data[i][j])


In [394]:
#adding game info
print(len(X))
print(len(factors))
for g in lol_data:
    blood,turret,inhib,baron,dragon,harold,win = g[5],g[6],g[7],g[8],g[9],g[10],g[4]
    temp=[blood,turret,inhib,baron,dragon,harold]
    #check who got the most amount of firsts
    t1=0
    t2=0
    for i in temp:
        if i ==1:
            t1+=1
        elif i==2:
            t2+=1
    if t1 == t2:
        continue
    #if P1>P2: P1,P2,win=P2,P1,-win  # (need to make player IDs sorted...)
    
    #for every combination of values, there are two teams and one won.
    #each X represents wether or not they got a first objective.
    #it will always be in the order thats in the csv (fb=5,ft=6,fi=7,fbaron=8,fd=9,fh=10)
    #factors.append(gm.Factor([X[blood],X[turret]], Pwin if win>0 else 1-Pwin) )
        
    factors.append(gm.Factor([X[t1],X[t2]], Pwin if win==1 else 1-Pwin))

7
7


In [395]:
# for i in factors:

#     for j in i.table:
#         if type(j) == np.float64:
#             continue
#         elif 0 in j:
#             print(table)

In [396]:
model = gm.GraphModel(factors)
print("its working")
#model.makeMinimal()  # merge any duplicate factors (e.g., repeated games)

its working


In [397]:
if model.nvar < 0:       # for very small models, we can do brute force inference:
    jt = model.joint()
    jt /= jt.sum()       # normalize the distribution and marginalize the table
    bel = [jt.marginal([i]) for i in range(nplayers)] 
else:                    # otherwise we need to use some approximate inference:
    from pyGM.messagepass import LBP, NMF
    #lnZ,bel = LBP(model, maxIter=10, verbose=True)   # loopy BP
    lnZ,bel = NMF(model, maxIter=3, verbose=True)  # Mean field

Iter 0: -35945.75858508854
Iter 1: -32344.25515811072
Iter 2: -32316.29118536767
Iter 3: -32316.29118536767


In [403]:
for i in bel:
    print(i.table)
i,j = 1,3

print("Estimated probability P{} beats P{} next time:".format(i,j))
# Expected value (over skill of P0, P1) of Pr[win | P0-P1]


if i<j:
    print( (bel[i]*bel[j]*gm.Factor([X[i],X[j]],Pwin)).table.sum() )
else:
    print( (bel[i]*bel[j]*gm.Factor([X[i],X[j]],1-Pwin)).table.sum() )

[0.00000000e+000 0.00000000e+000 1.82319904e-310 7.79235611e-141
 4.13979949e-037 1.00000000e+000 8.16880266e-032 3.49196486e-129
 2.04087814e-290 0.00000000e+000]
[0.00000000e+000 0.00000000e+000 3.18232827e-219 6.14947925e-070
 1.00000000e+000 7.16539667e-013 4.65194809e-106 2.08923878e-278
 0.00000000e+000 0.00000000e+000]
[0.00000000e+000 1.24515787e-249 2.14470815e-106 8.00396001e-023
 1.00000000e+000 9.74015027e-041 2.90666508e-142 1.58473476e-303
 0.00000000e+000 0.00000000e+000]
[0.00000000e+000 6.77970260e-254 1.04442183e-114 9.82232349e-030
 1.00000000e+000 2.26006744e-028 3.03837168e-112 7.87988032e-251
 0.00000000e+000 0.00000000e+000]
[0.00000000e+000 7.82634545e-311 1.18585444e-136 1.76349209e-032
 1.00000000e+000 1.93494384e-042 3.16871749e-157 0.00000000e+000
 0.00000000e+000 0.00000000e+000]
[0.00000000e+000 9.81189485e-305 4.49470448e-160 4.02725436e-060
 1.09641092e-006 9.99998904e-001 2.82749234e-042 1.93651086e-130
 1.81233462e-263 0.00000000e+000]
[3.21911091e-178